In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.rendered_html { font-size: 16px; }</style>"))
import warnings
warnings.filterwarnings('ignore')

# Python + Jupyter introduction with a CG Trajectory

This notebook is a first approach to show how an analysis using python libraries in a jupyter notebook looks like.

-------------

### First tasks as example:

   - Showing how to load a trajectory and how to access to the frame and topology attributes.
   - Showing how to execute methods.
   - Showing how to make a simple plot.
   - Showing how to make a simple analysis.
   - Showing how to visualize a trajectory on the same jupyter notebook.
   - Writing a trajectory with only the MOP beads coordinates.

## Importing Libraries

In [ ]:
import mdtraj as md
import seaborn as sns
import matplotlib.pyplot as plt
import nglview as nv

## Loading a frame with MDTraj

The xtc original trajectory seems quite heavy. Lets try to check the info contained in the first frame and topology:

In [ ]:
pdb_file  = 'cell.pdb'
traj_file = 'cell.xtc'
top_file  = 'topol.tpr'

In [ ]:
traj = md.load(traj_file, top=pdb_file, frame=0)

In [ ]:
print(traj)

Two useful python commands to deal with classes, methods and attributes: `help()` and `dir()`

In [ ]:
help(traj)

In [ ]:
dir(traj)

Lets have a look to some attributes of the class mdtraj.Trajectory such as the xyz coordinates or the unit cell vectors.

In [ ]:
traj.xyz

The trajectory is a numpy array. Numpy has become the standard maths library with "simple" algebraic methods. More sophisticated functions can be found in libraries as scipy, pandas, scikit-learn, etc.

In [ ]:
traj.xyz.shape

In [ ]:
traj.xyz[0,100,:]

In [ ]:
print(traj.xyz[0,:,0].min(), traj.xyz[0,:,0].max())

In [ ]:
print(traj.xyz[0,:,0].mean(), traj.xyz[0,:,1].mean(), traj.xyz[0,:,2].mean())

In [ ]:
traj.xyz.mean(axis=1)

In [ ]:
traj.unitcell_vectors

Classes can store classes inside, like matryoshka nested dolls.

In [ ]:
traj.topology

In [ ]:
help(traj.topology)

In [ ]:
dir(traj.topology)

In [ ]:
print(traj.topology.n_atoms, 'atoms in', traj.topology.n_residues, 'residues in', traj.topology.n_chains, 'chain.')

In [ ]:
list(traj.topology.residues)

Classes not only stores attributes but methods as well. I am sure you were already wondering how to make atoms selections using the mdtraj.Topology:

In [ ]:
beads_protein_indices = traj.topology.select('protein')

In [ ]:
print(beads_protein_indices)

In [ ]:
print(len(beads_protein_indices),'beads selected')

In [ ]:
traj.topology.residue(1000)

In [ ]:
a_single_popc = traj.topology.select('resid 1001 and resname POPC')

In [ ]:
a_single_popc

With a subset of atom indices in a list, a new trajectory object can be extracted corresponding to the selected atoms info only.

In [ ]:
a_single_popc_traj = traj.atom_slice(a_single_popc)

In [ ]:
a_single_popc_traj

In [ ]:
a_single_popc_traj.topology

It is now time to see how we can embed an interactive view of a molecule in the jupyter notebook.

In [ ]:
view = nv.show_mdtraj(a_single_popc_traj)
view.clear()
view.add_ball_and_stick(radius=2)
view

Lets try now to do something just a bit more complicated to see how cool is working this way. Lets define a subsection in this notebook as if this were a real working report.

--------

## Density profile of water molecules and ions in the box

Is there any water molecule or ion inside the membrane?
Lets take to start the analysis the z coordinate of the POPC beads named PO4 and NC3 to define the outer frontier of the bilayer leaflets.

Note: Only the first 100 frames will be considered to make the example computationally light.

In [ ]:
selection_heads = traj.topology.select("name PO4 or name NC3")

In [ ]:
len(selection_heads)

In [ ]:
iter_heads = md.iterload(traj_file, top=pdb_file, chunk=100, atom_indices=selection_heads)

In [ ]:
heads_frames_0_to_100 = next(iter_heads)

In [ ]:
heads_zcoor = heads_frames_0_to_100.xyz[:,:,2]

In [ ]:
print(heads_zcoor)

In [ ]:
print(heads_zcoor.shape)

In [ ]:
print(heads_zcoor.flatten())

In [ ]:
print(heads_zcoor.flatten().shape)

In [ ]:
heads_zcoor = heads_zcoor.flatten()

In [ ]:
sns.kdeplot(heads_zcoor, shade=True, vertical=True, color=sns.xkcd_rgb["peach"])
plt.title("Density of lipids heads")
plt.show()

In [ ]:
%matplotlib notebook
#%matplotlib inline

In [ ]:
plt.close()

Lets do now the same with waters and ions

In [ ]:
waters = traj.topology.select('resname PW')
ions   = traj.topology.select('resname ION')

iter_waters = md.iterload(traj_file, top=pdb_file, chunk=100, atom_indices=waters)
iter_ions   = md.iterload(traj_file, top=pdb_file, chunk=100, atom_indices=ions)

waters_zcoor = next(iter_waters).xyz[:,:,2].flatten()
ions_zcoor   = next(iter_waters).xyz[:,:,2].flatten()

In [ ]:
sns.kdeplot(heads_zcoor, shade=True, vertical=True, color=sns.xkcd_rgb["peach"], label='POPC heads')
sns.kdeplot(waters_zcoor, shade=True, vertical=True, color=sns.xkcd_rgb["light blue"], label='Waters')
sns.kdeplot(ions_zcoor, vertical=True, color=sns.xkcd_rgb["red"], label='Ions')
plt.title("Density of lipids heads")
plt.show()

In [ ]:
plt.close()

-----

# Just one thing left

Finnally, in order to keep on working with the study of the structural observables of the protein only, lets write for the future a new trajectory stripping out lipids, waters and ions.

In [ ]:
beads_protein_indices = traj.topology.select('protein')

In [ ]:
mop_traj = md.load(traj_file, top=pdb_file, atom_indices=beads_protein_indices)

In [ ]:
mop_traj

In [ ]:
mop_traj.save('MOP.h5')

In [ ]:
aa= md.load('MOP.h5')

And remember... we have a trajectory viewer!!

In [ ]:
view = nv.show_mdtraj(aa)
view.clear()
view.add_ball_and_stick(radius=1)
view

# And in case you are not in love with jupyter notebook yet...

This jupyter notebook can be downloaded as html, latex, pdf... or even automatically turned into a slides show!

Note:
```bash
jupyter nbconvert First_Approach.ipynb --to slides --post serve
```